# Content-Based Recommendations

Here, a recommendation system is built that will provide the top ten most similar titles to a given input book using that dataset that was cleaned in the Data Preprocessing and EDA notebook.

In [1]:
# Import necessary libraries

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load and preview cleaned_df 

cleaned_df = pd.read_csv('cleaned_df.csv')
cleaned_df.head()

,user_id,book_id,rating,authors,title,language_code,average_rating,tag
0,1,258,5,"Carlos Ruiz Zafón, Lucia Graves",The Shadow of the Wind (The Cemetery of Forgot...,eng,4.24,wish-list
1,2,4081,4,Tom Wolfe,I am Charlotte Simmons,en-US,3.40,young-adult
2,2,260,5,Dale Carnegie,How to Win Friends and Influence People,eng,4.13,تنمية-بشرية
3,2,9296,5,"Alice Miller, Ruth Ward",The Drama of the Gifted Child: The Search for ...,en-GB,4.09,wish-list
4,2,2318,3,"Thomas J. Stanley, William D. Danko",The Millionaire Next Door: The Surprising Secr...,eng,4.00,wish-list


To determine how similar one book is to another, the tag name and author(s) name(s) will be used.

In [3]:
# Create dataframe with features to be used to check for similarity, 'tag', 'authors'

data = cleaned_df[['title','tag','authors']]
data.head()

,title,tag,authors
0,The Shadow of the Wind (The Cemetery of Forgot...,wish-list,"Carlos Ruiz Zafón, Lucia Graves"
1,I am Charlotte Simmons,young-adult,Tom Wolfe
2,How to Win Friends and Influence People,تنمية-بشرية,Dale Carnegie
3,The Drama of the Gifted Child: The Search for ...,wish-list,"Alice Miller, Ruth Ward"
4,The Millionaire Next Door: The Surprising Secr...,wish-list,"Thomas J. Stanley, William D. Danko"


In [4]:
# View number of titles and review counts of each

data.groupby(['title']).count() #9964 titles

,tag,authors
title,,
"Angels (Walsh Family, #3)",263,263
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",113,113
#GIRLBOSS,195,195
'Salem's Lot,4433,4433
"'Tis (Frank McCourt, #2)",703,703
"1,000 Places to See Before You Die",398,398
1/4 جرام,196,196
"10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works",297,297
"100 Bullets, Vol. 1: First Shot, Last Call",170,170


Since each book was assigned the most used tag name for that book, each observation of the same book will have the same tag name and author name. Duplicates can be removed to save computation time.

In [5]:
# We only need one account of each book
# Drop duplicates, duplicate tags and authors are all the same

data.drop_duplicates(subset ="title", keep="first",inplace=True)
data.shape

/Users/erikadauria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(9964, 3)

In [6]:
# Prepare dataframe so tags and authors can be read as one single string

data['title'] = data['title'].map(lambda x: x if type(x)!=str else x.lower())
data['tag'] = data['tag'].map(lambda x: x if type(x)!=str else x.lower())
data['authors'] = data['authors'].map(lambda x: x if type(x)!=str else x.lower())
data['authors'] = data['authors'].map(lambda x: x.replace(" ",""))
data['authors'] = data['authors'].map(lambda x: x.replace(","," "))
data['words_of_interest'] = data.apply(lambda row: row.tag + " " + row.authors, axis=1)
data = data.drop(columns=['tag','authors'])
data = data.set_index('title') 
data.head()

/Users/erikadauria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/erikadauria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/erikadauria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,words_of_interest
title,
"the shadow of the wind (the cemetery of forgotten books, #1)",wish-list carlosruizzafón luciagraves
i am charlotte simmons,young-adult tomwolfe
how to win friends and influence people,تنمية-بشرية dalecarnegie
the drama of the gifted child: the search for the true self,wish-list alicemiller ruthward
the millionaire next door: the surprising secrets of americas wealthy,wish-list thomasj.stanley williamd.danko


In [7]:
# Instantiate CountVectorizer() and compute cosine similarity matrix

cv = CountVectorizer()
cv_matrix = cv.fit_transform(data['words_of_interest'])
cosine_sim = cosine_similarity(cv_matrix, cv_matrix)
indices = pd.Series(data.index)

In [8]:
# Create function to provide top ten books similar to the input

def content_based_recommendations(title):
    recommended_books = []
    idx = indices[indices == title.lower()].index[0]
    sim_scores = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10 = list(sim_scores.iloc[1:11].index)
    for i in top_10:
        recommended_books.append(list(data.index)[i])    
    return recommended_books

Now that there is a function to compute the top ten most similar items to an input title, we will try it out with a few titles.

In [9]:
# Recommend books similar to 'The Hunger Games (The Hunger Games, #1)'

content_based_recommendations('The Hunger Games (The Hunger Games, #1)')

['catching fire (the hunger games, #2)',
 'the hunger games (the hunger games, #1)',
 'mockingjay (the hunger games, #3)',
 'the rithmatist (the rithmatist, #1)',
 'if there be thorns (dollanganger, #3)',
 'angelfire (angelfire, #1)',
 'mosquitoland',
 'mitosis (the reckoners, #1.5)',
 'the power of six (lorien legacies, #2)',
 'frostbite (vampire academy, #2)']

In [10]:
# Recommend books similar to 'The Hobbit'

content_based_recommendations('The Hobbit')

['the hobbit',
 'the two towers (the lord of the rings, #2)',
 'the return of the king (the lord of the rings, #3)',
 'the lord of the rings (the lord of the rings, #1-3)',
 'the complete guide to middle-earth',
 'the children of húrin',
 'the silmarillion (middle-earth universe)',
 'vicious (sinners of saint, #1)',
 'harry potter page to screen: the complete filmmaking journey',
 'remember me?']

In [11]:
# Recommend books similar to 'Pride and Prejudice

content_based_recommendations('Pride and Prejudice')

['vicious (sinners of saint, #1)',
 'love & misadventure',
 'the cellist of sarajevo',
 'john dies at the end (john dies at the end, #1)',
 'in the afterlight (the darkest minds, #3)',
 'the man who walked between the towers',
 'adam',
 'succubus blues (georgina kincaid, #1)',
 'breath',
 "william shakespeare's star wars: verily, a new hope (william shakespeare's star wars, #4)"]

In [13]:
cleaned_df

,user_id,book_id,rating,authors,title,language_code,average_rating,tag
0,1,258,5,"Carlos Ruiz Zafón, Lucia Graves",The Shadow of the Wind (The Cemetery of Forgot...,eng,4.24,wish-list
1,2,4081,4,Tom Wolfe,I am Charlotte Simmons,en-US,3.40,young-adult
2,2,260,5,Dale Carnegie,How to Win Friends and Influence People,eng,4.13,تنمية-بشرية
3,2,9296,5,"Alice Miller, Ruth Ward",The Drama of the Gifted Child: The Search for ...,en-GB,4.09,wish-list
4,2,2318,3,"Thomas J. Stanley, William D. Danko",The Millionaire Next Door: The Surprising Secr...,eng,4.00,wish-list
5,2,26,4,Dan Brown,"The Da Vinci Code (Robert Langdon, #2)",eng,3.79,روايات
6,2,315,3,"Spencer Johnson, Kenneth H. Blanchard",Who Moved My Cheese?,eng,3.74,روانشناسی
7,2,33,4,Arthur Golden,Memoirs of a Geisha,eng,4.08,wwii
8,2,301,5,Joseph Conrad,Heart of Darkness,eng,3.42,victorian
9,2,2686,5,"W. Chan Kim, Renée Mauborgne",Blue Ocean Strategy: How To Create Uncontested...,NaN,3.86,work-related


In [ ]:
cle